# sveltejs/svelte

> 

In [ ]:
#| default_exp sveltejs-svelte

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()
from nbdev.showdoc import *

In [ ]:
#| export
from lovely_docs.utils import settings, GitSource, clone_repo
import llm
from llm.models import Usage
from pydantic import BaseModel, Field
from jinja2 import Environment, FileSystemLoader
import asyncio
import json
import shutil
from pathlib import Path
import langsmith as ls
import logging

In [ ]:
#| export

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

logger.debug("hello")

+0.094s DEBUG <ipykernel>:6 <module> hello


In [ ]:
#| export

source = settings.sources["sveltejs/svelte"]
assert isinstance(source, GitSource)

commit = clone_repo(source)
source.commit = commit # Replace the literal commit (master) with the hash
clone_dir = settings.git_dir / source.name

In [ ]:
#| export

doc_dir_path = clone_dir / source.doc_dir

def get_files_from_fs(root):
    files = []
    for path in root.rglob('*'):
        if path.is_file():
            rel_path = path.relative_to(root)
            depth = len(rel_path.parts) - 1
            files.append((depth, str(rel_path)))
    files.sort(key=lambda x: (x[0], x[1]))
    return [f[1] for f in files]

files = get_files_from_fs(doc_dir_path)

In [ ]:
files[:10]

['index.md',
 '01-introduction/01-overview.md',
 '01-introduction/02-getting-started.md',
 '01-introduction/03-svelte-files.md',
 '01-introduction/04-svelte-js-files.md',
 '01-introduction/index.md',
 '02-runes/01-what-are-runes.md',
 '02-runes/02-$state.md',
 '02-runes/03-$derived.md',
 '02-runes/04-$effect.md']

In [ ]:
#| export

model = llm.get_async_model(settings.model)
model.key = settings.api_key

In [ ]:
#| export
class PageReplySchema(BaseModel):
    better_name: str = Field(description="")
    digest: str = Field(title="Digest, format: markdown", )
    short_digest: str = Field(title="Short digest, format:markdown")
    essence: str = Field(title="Essence, format:txt")
    relevant: bool


In [ ]:
#| export
jinja_env = Environment(loader=FileSystemLoader(settings.templates_dir))

class DocPage(BaseModel):
    """Represents a single documentation page"""
    path: Path
    fulltext: str
    name: str = ""
    digest: str = ""
    short_digest: str = ""
    essence: str = ""
    relevant: bool = True
    usage: Usage|None = None

@ls.traceable("chain", name="Process page")
async def llm_process_page(page: DocPage, libname: str, fake=True) -> DocPage:
    logger.debug(f"Processing {page.path}")

    # Load and render template
    template = jinja_env.get_template("process_page.j2")
    inputs = {
        "text": page.fulltext,
        "filename": page.path.name,
        "path": page.path.parent.name + "/",
        "libname": libname
    }
    with ls.trace("Template", "prompt", inputs=inputs) as trace:
        prompt = template.render(**inputs)
        trace.end(outputs=prompt)

    if fake:
        reply = PageReplySchema(
            better_name=f"{page.path.name}-improved",
            digest="summary_markdown",
            short_digest="short_summary_markdown",
            essence="one line",
            relevant=False if page.path.name == "index.md" else True
        )
        usage = Usage(12, 34)
    else:
        with ls.trace("LLM call", "llm", inputs={"prompt": prompt}) as trace:
            res = await (await model.prompt(prompt=prompt, schema=PageReplySchema))
            trace.end(outputs=await res.text())

        with ls.trace("Parse", "parser", inputs={"input": await res.text()}) as trace:
            reply = PageReplySchema.model_validate_json(await res.text())
            # Normalize better_name: lowercase, replace spaces with hyphens, remove .md extension
            reply.better_name = reply.better_name.lower().replace(' ', '-').removesuffix('.md')
            trace.end(outputs=reply)
        usage = await res.usage()

    return DocPage(
        path=page.path,
        fulltext=page.fulltext,
        name=reply.better_name,
        digest=reply.digest,
        short_digest=reply.short_digest,
        essence=reply.essence,
        relevant=reply.relevant,
        usage=usage)

In [ ]:
res = await llm_process_page(
    page=DocPage(path=files[1], fulltext=(settings.git_dir/source.name/source.doc_dir/files[1]).read_text()),
    libname=source.name, fake=False)

print(res.model_dump_json(indent=2))

+0.600s DEBUG <ipykernel>:17 llm_process_page Processing 01-introduction/01-overview.md


{
  "path": "01-introduction/01-overview.md",
  "fulltext": "---\ntitle: Overview\n---\n\nSvelte is a framework for building user interfaces on the web. It uses a compiler to turn declarative components written in HTML, CSS and JavaScript...\n\n```svelte\n<!--- file: App.svelte --->\n<script>\n\tfunction greet() {\n\t\talert('Welcome to Svelte!');\n\t}\n</script>\n\n<button onclick={greet}>click me</button>\n\n<style>\n\tbutton {\n\t\tfont-size: 2em;\n\t}\n</style>\n```\n\n...into lean, tightly optimized JavaScript.\n\nYou can use it to build anything on the web, from standalone components to ambitious full stack apps (using Svelte's companion application framework, [SvelteKit](../kit)) and everything in between.\n\nThese pages serve as reference documentation. If you're new to Svelte, we recommend starting with the [interactive tutorial](/tutorial) and coming back here when you have questions.\n\nYou can also try Svelte online in the [playground](/playground) or, if you need a more fu

In [ ]:
#| export

class DocDirectory(BaseModel):
    """Represents a directory in the documentation structure"""
    path: Path
    name: str = ""
    digest: str = ""
    essence: str = ""
    pages: list[DocPage] = Field(default_factory=list)
    subdirs: list['DocDirectory'] = Field(default_factory=list)
    relevant: bool = True
    usage: Usage | None = None

def build_markdown_doc_tree(root:Path, path:Path = Path()) -> DocDirectory:
    assert root.exists() and root.is_dir()
    assert (root/path).exists() and (root/path).is_dir()

    doc_dir = DocDirectory(path=path)

    # Get immediate children only
    for item in sorted((root/path).iterdir()):
        if item.is_file() and item.suffix == '.md':
            # We'll process files later, just record them
            doc_dir.pages.append(DocPage(path=item.relative_to(root), fulltext=item.read_text()))
        if item.is_dir():
            doc_dir.subdirs.append(build_markdown_doc_tree(root, item.relative_to(root)))

    return doc_dir

In [ ]:
doc = settings.git_dir / source.name / source.doc_dir
tree = build_markdown_doc_tree(doc)
print(tree.model_dump_json(indent=2))

{
  "path": ".",
  "name": "",
  "digest": "",
  "essence": "",
  "pages": [
    {
      "path": "index.md",
      "fulltext": "---\ntitle: Svelte\n---\n",
      "name": "",
      "digest": "",
      "short_digest": "",
      "essence": "",
      "relevant": true,
      "usage": null
    }
  ],
  "subdirs": [
    {
      "path": "01-introduction",
      "name": "",
      "digest": "",
      "essence": "",
      "pages": [
        {
          "path": "01-introduction/01-overview.md",
          "fulltext": "---\ntitle: Overview\n---\n\nSvelte is a framework for building user interfaces on the web. It uses a compiler to turn declarative components written in HTML, CSS and JavaScript...\n\n```svelte\n<!--- file: App.svelte --->\n<script>\n\tfunction greet() {\n\t\talert('Welcome to Svelte!');\n\t}\n</script>\n\n<button onclick={greet}>click me</button>\n\n<style>\n\tbutton {\n\t\tfont-size: 2em;\n\t}\n</style>\n```\n\n...into lean, tightly optimized JavaScript.\n\nYou can use it to build a

In [ ]:
#| export

class DirReplySchema(BaseModel):
    better_name: str
    digest: str = Field(title="Directory digest, fmt:markdown")
    essence: str
    relevant: bool

@ls.traceable("chain", name="Process directory")
async def llm_process_directory(directory: DocDirectory,  libname: str) -> DocDirectory:
    """Create a summary for a directory based on its relevant pages and subdirectories"""

    # If the directory did not have any relevant pages / subdirs, we should not be called.
    assert any(x for x in directory.pages+directory.subdirs if x.relevant)

    logger.debug(f"Processing {directory.path}")

    template = jinja_env.get_template("process_directory.j2")
    input = {
        "dirname": directory.path.name,
        "path": directory.path.parent.name,
        "pages": [p for p in directory.pages if p.relevant],
        "subdirs": [s for s in directory.subdirs if s.relevant],
        "libname": libname
    }
    with ls.trace("Template", "prompt", inputs=input) as trace:
        prompt = template.render(**input)
        trace.end(outputs=prompt)

    with ls.trace("LLM call", "llm", inputs={"prompt": prompt}) as trace:
        res = await (await model.prompt(prompt=prompt, schema=DirReplySchema))
        trace.end(outputs=await res.text())
        usage = await res.usage()

    with ls.trace("Parse", "parser", inputs={"input": await res.text()}) as trace:
        reply = DirReplySchema.model_validate_json(await res.text())
        trace.end(outputs=reply)

    res = directory.model_copy(deep=True)
    res.name = reply.better_name
    res.digest = reply.digest
    res.essence = reply.essence
    res.relevant = reply.relevant
    res.usage = usage
    return res

In [ ]:
@ls.traceable("chain", name="Process directory tree")
async def process_tree_depth_first(doc_dir: DocDirectory, libname: str) -> DocDirectory:
    """
    Process documentation tree depth-first with parallel processing.
    Mutates the doc_dir object.
    """

    # First, recursively process all subdirectories in parallel
    subdirs: list[DocDirectory] = await asyncio.gather(*[
        process_tree_depth_first(subdir, libname) for subdir in doc_dir.subdirs
    ])
    subdirs = sorted(subdirs, key=lambda s: s.path)

    # Then process all pages in this directory in parallel
    pages = await asyncio.gather(*[
        llm_process_page(page, libname, False) for page in doc_dir.pages
    ])
    pages = sorted(pages, key=lambda s: s.path)

    if not any(x for x in subdirs+pages if x.relevant):
        return DocDirectory(path=doc_dir.path, relevant=False)

    return await llm_process_directory(
        directory=DocDirectory(path=doc_dir.path, pages=pages, subdirs=subdirs),
        libname=libname)

In [ ]:
tree.subdirs

[DocDirectory(path=Path('01-introduction'), name='', digest='', essence='', pages=[DocPage(path=Path('01-introduction/01-overview.md'), fulltext="---\ntitle: Overview\n---\n\nSvelte is a framework for building user interfaces on the web. It uses a compiler to turn declarative components written in HTML, CSS and JavaScript...\n\n```svelte\n<!--- file: App.svelte --->\n<script>\n\tfunction greet() {\n\t\talert('Welcome to Svelte!');\n\t}\n</script>\n\n<button onclick={greet}>click me</button>\n\n<style>\n\tbutton {\n\t\tfont-size: 2em;\n\t}\n</style>\n```\n\n...into lean, tightly optimized JavaScript.\n\nYou can use it to build anything on the web, from standalone components to ambitious full stack apps (using Svelte's companion application framework, [SvelteKit](../kit)) and everything in between.\n\nThese pages serve as reference documentation. If you're new to Svelte, we recommend starting with the [interactive tutorial](/tutorial) and coming back here when you have questions.\n\nYou 

In [ ]:
tree.subdirs = tree.subdirs[:2]
print(tree.model_dump_json(indent=2))

{
  "path": ".",
  "name": "",
  "digest": "",
  "essence": "",
  "pages": [
    {
      "path": "index.md",
      "fulltext": "---\ntitle: Svelte\n---\n",
      "name": "",
      "digest": "",
      "short_digest": "",
      "essence": "",
      "relevant": true,
      "usage": null
    }
  ],
  "subdirs": [
    {
      "path": "01-introduction",
      "name": "",
      "digest": "",
      "essence": "",
      "pages": [
        {
          "path": "01-introduction/01-overview.md",
          "fulltext": "---\ntitle: Overview\n---\n\nSvelte is a framework for building user interfaces on the web. It uses a compiler to turn declarative components written in HTML, CSS and JavaScript...\n\n```svelte\n<!--- file: App.svelte --->\n<script>\n\tfunction greet() {\n\t\talert('Welcome to Svelte!');\n\t}\n</script>\n\n<button onclick={greet}>click me</button>\n\n<style>\n\tbutton {\n\t\tfont-size: 2em;\n\t}\n</style>\n```\n\n...into lean, tightly optimized JavaScript.\n\nYou can use it to build a

In [ ]:
# Process the entire documentation tree
res = await process_tree_depth_first(tree, source.name)

+3.786s DEBUG <ipykernel>:17 llm_process_page Processing 01-introduction/02-getting-started.md
+0.007s DEBUG <ipykernel>:17 llm_process_page Processing 01-introduction/04-svelte-js-files.md
+0.010s DEBUG <ipykernel>:17 llm_process_page Processing 01-introduction/03-svelte-files.md
+0.008s DEBUG <ipykernel>:17 llm_process_page Processing 01-introduction/index.md
+0.008s DEBUG <ipykernel>:17 llm_process_page Processing 01-introduction/01-overview.md
+0.007s DEBUG <ipykernel>:17 llm_process_page Processing 02-runes/04-$effect.md
+0.007s DEBUG <ipykernel>:17 llm_process_page Processing 02-runes/01-what-are-runes.md
+0.008s DEBUG <ipykernel>:17 llm_process_page Processing 02-runes/06-$bindable.md
+0.007s DEBUG <ipykernel>:17 llm_process_page Processing 02-runes/05-$props.md
+0.008s DEBUG <ipykernel>:17 llm_process_page Processing 02-runes/03-$derived.md
+0.007s DEBUG <ipykernel>:17 llm_process_page Processing 02-runes/02-$state.md
+0.008s DEBUG <ipykernel>:17 llm_process_page Processing 02-

In [ ]:
def calculate_total_usage(doc_dir: DocDirectory) -> Usage:
    """Calculate total usage for a directory tree including all pages, subdirs, and summaries"""
    total_input = 0
    total_output = 0

    # Add usage from all pages in this directory
    for page in doc_dir.pages:
        total_input += page.usage.input or 0
        total_output += page.usage.output or 0

    # Add usage from directory summarization
    if doc_dir.usage:
        total_input += doc_dir.usage.input or 0
        total_output += doc_dir.usage.output or 0

    # Recursively add usage from subdirectories
    for subdir in doc_dir.subdirs:
        subdir_usage = calculate_total_usage(subdir)
        total_input += subdir_usage.input or 0
        total_output += subdir_usage.output or 0

    return Usage(input=total_input, output=total_output)


In [ ]:
# Calculate total usage for the entire tree
total_usage = calculate_total_usage(res)
print(f"\nTotal Usage:")
print(f"  Input tokens: {total_usage.input:,}")
print(f"  Output tokens: {total_usage.output:,}")
print(f"  Total tokens: {(total_usage.input + total_usage.output):,}")


Total Usage:
  Input tokens: 38,161
  Output tokens: 8,141
  Total tokens: 46,302


In [ ]:
def save_doc_directory(path: Path, doc: DocDirectory):
    if path.exists(): shutil.rmtree(path)
    path.mkdir(parents=True, exist_ok=True)

    (path/"digest.md").write_text(doc.digest)
    (path/"essence.md").write_text(doc.essence)
    for page in doc.pages:
        (path/page.name).mkdir(exist_ok=True)
        (path/page.name/"essence.md").write_text(page.essence)
        (path/page.name/"fulltext.md").write_text(page.fulltext)
        (path/page.name/"digest.md").write_text(page.digest)
        (path/page.name/"short_digest.md").write_text(page.short_digest)

    for subdir in doc.subdirs:
        save_doc_directory(path/subdir.name, subdir)

In [ ]:
save_doc_directory(settings.output_dir / source.name, res)

In [ ]:
res.subdirs[0]

DocDirectory(path=Path('01-introduction'), name='Introduction', digest='## Getting Started\n\nInitialize projects with `npx sv create myapp` (SvelteKit + Vite) or `npm create vite@latest` with Svelte template. VS Code extension and `sv check` command available for development.\n\n## Component Structure\n\nSvelte components use `.svelte` files with optional sections:\n```svelte\n<script module>// module-level logic</script>\n<script>// instance logic</script>\n<!-- markup -->\n<style>/* auto-scoped CSS */</style>\n```\n\n- `<script>`: Instance variables and logic, runes for props/reactivity\n- `<script module>`: Module-level exports, runs once\n- `<style>`: Component-scoped CSS\n\n## Reactive Logic\n\n`.svelte.js` and `.svelte.ts` files support runes for creating reusable reactive logic and shared state.', essence='Foundation for setting up Svelte projects, understanding component file structure, and creating reactive logic with runes.', pages=[DocPage(path=Path('01-introduction/01-over

In [ ]:
def file_map(doc: DocDirectory):
    filemap = {}

    # The filemap is keyed by original name so it's easy to get the pages in the original order.

    for subdir in doc.subdirs:
        filemap |= file_map(subdir)
        filemap[str(subdir.path)] = subdir.model_dump(include=["name", "relevant", "usage"]) | {"type": "directory"}

    for page in doc.pages:
        filemap[str(page.path)] = page.model_dump(include=["name", "relevant", "usage"]) | {"type": "page"}
    return filemap

In [ ]:
from lovely_docs.utils import Source, WebSource

def build_metadata(source: Source, doc: DocDirectory):
    if isinstance(source, GitSource):
        source_type = "git"
    elif isinstance(source, WebSource):
        source_type = "web"
    else:
        raise TypeError(f"Unknown source type: {type(source)}")
    return {
        "map": file_map(doc),
        "source_type": source_type,
        "source": source.model_dump(mode="json")
    }



In [ ]:
(settings.output_dir / source.name / "index.json").write_text(json.dumps(build_metadata(source, res), indent=2))

3812